## Import all Libraries here

In [33]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_columns', 500)

#!pip install imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler 

## **Data Cleaning & Imputation**


In [34]:
ext_baseline3=pd.read_csv("ext_baseline3.csv")
ext_baseline3=ext_baseline3.drop(['Unnamed: 0','address(original)','size(original)','intercropCrop','cropVariety','machines(original)'],axis=1)
ext_baseline3=ext_baseline3.dropna(axis=0, how='all')
ext_baseline3=ext_baseline3.dropna(subset=["notPayupLoan"])
cleanup={"notPayupLoan":{"no":0,"yes":1}}
ext_baseline3=ext_baseline3.replace(cleanup)
ext_baseline3["numberOfLand"]=ext_baseline3["numberOfLand"].fillna(ext_baseline3["numberOfLand"].mean())
ext_baseline3["size"]=ext_baseline3["size"].fillna(ext_baseline3["size"].mean())
ext_baseline3["estimateMonthlyIncome"]=ext_baseline3["estimateMonthlyIncome"].fillna(ext_baseline3["estimateMonthlyIncome"].median())
ext_baseline3["applyLoanAmount"]=ext_baseline3["applyLoanAmount"].fillna(ext_baseline3["applyLoanAmount"].median())
ext_baseline3["yearsCultivating"]=ext_baseline3["yearsCultivating"].fillna(ext_baseline3["yearsCultivating"].median())
ext_baseline3["address"]=ext_baseline3["address"].fillna(ext_baseline3["address"].mode()[0])
ext_baseline3["intercropping"]=ext_baseline3["intercropping"].fillna(ext_baseline3["intercropping"].mode()[0])
ext_baseline3.address=ext_baseline3.address.replace({'Akasan taura ':'taura','Akasan taura':'taura'})
ext_baseline3.address=ext_baseline3.address.replace({'Gurjawa Kwalam ':'Gurjawa Kwalam'})  
ext_baseline3.crop=ext_baseline3.crop.replace({'Oil palm':'Oil Palm'})    
ext_baseline3.crop=ext_baseline3.crop.replace({'maize':'Maize'})    
ext_baseline3.machines=ext_baseline3.machines.replace({'n':'No','y':'Yes'})    
ext_baseline3.notPayupLoan=ext_baseline3.notPayupLoan.replace({'No':1,'Yes':0})    
ext_baseline3.notPayupLoan=ext_baseline3.notPayupLoan.astype('int')
ext_baseline3["notPayupLoan"].value_counts()

1    81
0     2
Name: notPayupLoan, dtype: int64

In [35]:
ext_baseline3.numberOfLand=ext_baseline3.numberOfLand.astype('int')
ext_baseline3.drop('size',axis=1,inplace=True)
ext_baseline3['income']=ext_baseline3.estimateMonthlyIncome.astype('int')
ext_baseline3.estimateMonthlyIncome[ext_baseline3.income<=50000]='below-50k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>50000)&(ext_baseline3.income<=100000)]='50k-100k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>100000)&(ext_baseline3.income<=250000)]='100k-250k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>250000)&(ext_baseline3.income<=500000)]='250k-500k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>500000)&(ext_baseline3.income<=1000000)]='500k-1m'
ext_baseline3.estimateMonthlyIncome[ext_baseline3.income>1000000]='1m-Above'
ext_baseline3.drop('income',axis=1,inplace=True)
ext_baseline3.estimateMonthlyIncome.value_counts()
ext_baseline3['amount']=ext_baseline3.applyLoanAmount.astype('int')
#ext_baseline3.applyLoanAmount[ext_baseline3.amount<=50000]='below-50k'
#ext_baseline3.applyLoanAmount[(ext_baseline3.amount>50000)&(ext_baseline3.amount<=100000)]='50k-100k'
#ext_baseline3.applyLoanAmount[(ext_baseline3.amount>100000)&(ext_baseline3.amount<=250000)]='100k-250k'
#ext_baseline3.applyLoanAmount[(ext_baseline3.amount>250000)&(ext_baseline3.amount<=500000)]='250k-500k'
#ext_baseline3.applyLoanAmount[(ext_baseline3.amount>500000)&(ext_baseline3.amount<=1000000)]='500k-1m'
#ext_baseline3.applyLoanAmount[ext_baseline3.amount>1000000]='1m-Above'
ext_baseline3.drop('amount',axis=1,inplace=True)
ext_baseline3['years']=ext_baseline3.yearsCultivating.astype('int')
ext_baseline3.yearsCultivating[ext_baseline3.years<=5]='below-5'
ext_baseline3.yearsCultivating[(ext_baseline3.years>5)&(ext_baseline3.years<=10)]='6-10'
ext_baseline3.yearsCultivating[(ext_baseline3.years>10)&(ext_baseline3.years<=20)]='11-20'
ext_baseline3.yearsCultivating[(ext_baseline3.years>20)&(ext_baseline3.years<=30)]='21-30'
ext_baseline3.yearsCultivating[(ext_baseline3.years>30)&(ext_baseline3.years<=40)]='31-40'
ext_baseline3.yearsCultivating[ext_baseline3.years>40]='41-Above'
#ext_baseline3.drop('applyLoanAmount',axis=1,inplace=True)
ext_baseline3.drop('address',axis=1,inplace=True)
ext_baseline3.drop('years',axis=1,inplace=True)

## Split Data before feature generation

In [36]:
from sklearn.model_selection import train_test_split
bl, bl_test = train_test_split(ext_baseline3, test_size=0.3, random_state=123)

In [37]:
bl_test.head()

,age,numberOfLand,owner_caretaker,crop,intercropping,machines,estimateMonthlyIncome,applyLoanAmount,yearsCultivating,notPayupLoan
103,45-54,1,Owner,Sorghum,Yes,Yes,below-50k,200000.0,11-20,1
109,25-34,3,Owner,Soya bean,No,Yes,100k-250k,1500000.0,below-5,1
7,45-54,2,Owner,sesame,Yes,Yes,50k-100k,250000.0,11-20,1
114,25-34,5,Owner,Cucumber,Yes,No,250k-500k,1600000.0,6-10,1
35,35-44,2,Inherited,Rice,No,Yes,50k-100k,200000.0,11-20,1


In [38]:
bl_test.columns

Index(['age', 'numberOfLand', 'owner_caretaker', 'crop', 'intercropping',
       'machines', 'estimateMonthlyIncome', 'applyLoanAmount',
       'yearsCultivating', 'notPayupLoan'],
      dtype='object')

## **Feature Engineering** 

In [39]:
def preprocess_yearsCultivating(df):
    cleanup={"yearsCultivating":{'Less than 5 years':1,"below-5":1,
                                 '5 to 10 years':2,"6-10":2,
                                "11-20":3,'10 to 15 years':3,'15 to 20 years':3,
                                "21-30":4,"31-40":4,"41-Above":4,'20 to 25 years':4,'25 years and more':4}}
    df=df.replace(cleanup)
    return(df)
def preprocess_estimateMonthlyIncome(df):
    cleanup={"estimateMonthlyIncome":{"below-50k":1,"50k-100k":1,"100k-250k":1,"250k-500k":1,'Less than 500,000':1,
                                      "500k-1m":2,'500,000 to 1,000,000':2,
                                      "1m-Above":3,'1,000,000 and more':3}}
    df=df.replace(cleanup)
    return(df)
#def preprocess_applyLoanAmount(df):
    #cleanup={"applyLoanAmount":{"below-50k":50,"50k-100k":100,"100k-250k":250,"250k-500k":500,"500k-1m":500,"1m-Above":1000}}
    #df=df.replace(cleanup)
    #return(df)
def preprocess_intercropping(df):
    cleanup={"intercropping":{"No":1,'Mixed cropping':1,
                              "Yes":2,'Intercropping':2}}
    df=df.replace(cleanup)
    return(df)
def preprocess_machines(df):
    cleanup={"machines":{"No":0,"Yes":2}}
    df=df.replace(cleanup)
    return(df)
def preprocess_owner_caretaker(df):
    cleanup={"owner_caretaker":{"Owner":3,
                                "Rental":1,"Lease":1,"Inherited":1,
                                "Caretaker":1}}
    df=df.replace(cleanup)
    return(df)

def preprocess_crop(df):
    cleanup={"crop":{"Cocoa":2,"sesame":2,"Sorghum":1,"Rice":1,'Food crops':1,'Cash crops':2,
                     "Millet":1,'Maize':1,'Tomato':1,'Oil Palm':2, 'Cassava':1}}
    df=df.replace(cleanup)
    crop=[1,2]
    for x in crop:
            df['crop'+str(x)]=(df.crop==x).astype('int')
    df.drop('crop',axis=1,inplace=True)
    return df

def preprocess_age(df):
    cleanup={"age":{"25-34":2,"25 or less":1,"35 to 45":3,"45 or more":4,'45-54':4,'65-Above':4,
                     "35-44":3,'55-64':4}}
    age=[1,2,3,4]
    for x in age:
            df['age'+str(x)]=(df.age==x).astype('int')
    df.drop('age',axis=1,inplace=True)
    return df
def preprocess_numberOfLand(df):
    df.numberOfLand[df.numberOfLand=='6-Above']=6
    df.numberOfLand[df.numberOfLand=='Less than 3 farms']=2
    df.numberOfLand[df.numberOfLand=='More than 3 farms']=3
    df.numberOfLand=df.numberOfLand.astype('int')
    df.numberOfLand=((df.numberOfLand>=3).astype('int'))+1
    return df

In [40]:
def feature_engineer(df):
    # Generate your features here!!!
    df.applyLoanAmount=df.applyLoanAmount.astype('int')
    df=preprocess_numberOfLand(df)
    df=preprocess_intercropping(preprocess_yearsCultivating(df))
    df=preprocess_machines(preprocess_estimateMonthlyIncome(df))
    df=preprocess_owner_caretaker(df)
    #df = preprocess_applyLoanAmount(df)
    df=preprocess_crop(df)
    df=preprocess_age(df)
    #df=preprocess_address(df)
    return df

In [41]:
cols=['age', 'number_of_land', 'owner_caretaker', 'crop',
            'intercropping', 'machines', 'estimate_monthly_income','years_cultivating','applyLoanAmount']

In [42]:
# run feature engineering on train set
bl = feature_engineer(bl)
bl.head()

,numberOfLand,owner_caretaker,intercropping,machines,estimateMonthlyIncome,applyLoanAmount,yearsCultivating,notPayupLoan,crop1,crop2,age1,age2,age3,age4
98,1,3,2,0,1,20000,3,1,0,1,0,0,0,0
59,1,3,1,2,1,160000,2,1,0,1,0,0,0,0
97,1,3,2,2,1,120000,3,1,1,0,0,0,0,0
26,1,1,2,2,1,300000,3,1,1,0,0,0,0,0
17,1,3,2,2,1,180000,4,1,1,0,0,0,0,0


## Train & Save Model (you can name it with your notebook's name)

In [43]:
bl=bl.rename({'numberOfLand':'number_of_land','estimateMonthlyIncome':'estimate_monthly_income','yearsCultivating':'years_cultivating','applyLoanAmount':'apply_loan_amount'},axis=1)

In [44]:
X=bl.drop(["notPayupLoan"],axis=1)
y=bl["notPayupLoan"]

In [45]:
from sklearn.linear_model import LogisticRegression
classifier =  LogisticRegression()
classifier.fit(X, y)
### Create a Pickle file using serialization 
import pickle
pickle_out = open("modelExample.pkl","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

## Load Test Set

In [46]:
# run feature engineering on test set
bl_test = feature_engineer(bl_test)

In [47]:
bl_test=bl_test.rename({'numberOfLand':'number_of_land','estimateMonthlyIncome':'estimate_monthly_income','yearsCultivating':'years_cultivating','applyLoanAmount':'apply_loan_amount'},axis=1)

In [48]:
X_test=bl_test.drop(["notPayupLoan"],axis=1)
y_test=bl_test["notPayupLoan"]

## Load Model

In [49]:
model = pickle.load(open('modelExample.pkl','rb'))

In [50]:
y_pred = model.predict_proba(X_test)[:,1]
y_p = model.predict(X_test)

In [51]:
X_test.columns

Index(['number_of_land', 'owner_caretaker', 'intercropping', 'machines',
       'estimate_monthly_income', 'apply_loan_amount', 'years_cultivating',
       'crop1', 'crop2', 'age1', 'age2', 'age3', 'age4'],
      dtype='object')

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score, roc_auc_score
#print(confusion_matrix(y_test,y_pred))

In [53]:
print(roc_auc_score(y_test, y_pred))

0.625


In [54]:
print(accuracy_score(y_test, y_p))

0.96
